In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import sklearn.tree
import sklearn.ensemble

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [13]:
#Data cleaning and classification
train_data_numeric = train_data.replace('male', 1)
train_data_numeric = train_data_numeric.replace('female', 0)

train_data_numeric = train_data_numeric.replace('Q', 2)
train_data_numeric = train_data_numeric.replace('S', 1)
train_data_numeric = train_data_numeric.replace('C', 0)

ageRange = train_data_numeric["Age"].max() - train_data_numeric["Age"].min()
def classifyAge(age):
    if not math.isnan(age):
        return int((age - train_data_numeric["Age"].min()) // (ageRange/5.0))
    else:
        return age
train_data_numeric["Age"] = train_data_numeric["Age"].apply(classifyAge) 


fareRange = train_data_numeric["Fare"].max() - train_data_numeric["Fare"].min()
def classifyFare(fare):
    if not math.isnan(fare):
        return int((fare - train_data_numeric["Fare"].min()) // (fareRange/4.0))
    else:
        return fare
train_data_numeric["Fare"] = train_data_numeric["Fare"].apply(classifyFare) 

def getFirstChar(s):
    if type(s) is not float:
        s = s.split(" ")
        s = s[0]
        return int(ord(s[0:1]))
    else:
        return s
train_data_numeric["CabinLetter"] = train_data_numeric["Cabin"].apply(getFirstChar)

def getCabinNumber(s):
    if type(s) is not float:
        s = s.split(" ")
        s = s[0]
        try:
            return int(s[1:])
        except:
            return float('nan')
    else:
        return s
train_data_numeric["CabinNumber"] = train_data_numeric["Cabin"].apply(getCabinNumber)
cabinNumberRange = train_data_numeric["CabinNumber"].max() - train_data_numeric["CabinNumber"].min()
def classifyCabinNumber(cabinNumber):
    if not math.isnan(cabinNumber):
        return int((cabinNumber - train_data_numeric["CabinNumber"].min()) // (cabinNumberRange/5.0))
    else:
        return cabinNumber
train_data_numeric["CabinNumber"] = train_data_numeric["CabinNumber"].apply(classifyCabinNumber) 




In [14]:
#Data cleaning and classification
test_data_numeric = test_data.replace('male', 1)
test_data_numeric = test_data_numeric.replace('female', 0)

test_data_numeric = test_data_numeric.replace('Q', 2)
test_data_numeric = test_data_numeric.replace('S', 1)
test_data_numeric = test_data_numeric.replace('C', 0)

ageRange = test_data_numeric["Age"].max() - test_data_numeric["Age"].min()
def classifyAge(age):
    if not math.isnan(age):
        return (age - test_data_numeric["Age"].min()) // (ageRange/4.0)
    else:
        return age
test_data_numeric["Age"] = test_data_numeric["Age"].apply(classifyAge) 


fareRange = test_data_numeric["Fare"].max() - test_data_numeric["Fare"].min()
def classifyFare(fare):
    if not math.isnan(fare):
        return int((fare - test_data_numeric["Fare"].min()) // (fareRange/7.0))
    else:
        return fare
test_data_numeric["Fare"] = test_data_numeric["Fare"].apply(classifyFare) 

def getFirstChar(s):
    if type(s) is not float:
        s = s.split(" ")
        s = s[0]
        return int(ord(s[0:1]))
    else:
        return s
test_data_numeric["CabinLetter"] = test_data_numeric["Cabin"].apply(getFirstChar)

def getCabinNumber(s):
    if type(s) is not float:
        s = s.split(" ")
        s = s[0]
        try:
            return int(s[1:])
        except:
            return float('nan')
    else:
        return s
test_data_numeric["CabinNumber"] = test_data_numeric["Cabin"].apply(getCabinNumber)
cabinNumberRange = test_data_numeric["CabinNumber"].max() - test_data_numeric["CabinNumber"].min()
def classifyCabinNumber(cabinNumber):
    if not math.isnan(cabinNumber):
        return (cabinNumber - test_data_numeric["CabinNumber"].min()) // (cabinNumberRange/5.0)
    else:
        return cabinNumber
test_data_numeric["CabinNumber"] = test_data_numeric["CabinNumber"].apply(classifyCabinNumber) 
test_data_numeric = test_data_numeric.replace(float('nan'), 6.0)


In [16]:
#Create model
train_data_numeric = train_data_numeric.replace(float('nan'), 6)
decide_columns = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked", "CabinLetter", "CabinNumber"]
dtree = sklearn.ensemble.RandomForestClassifier(n_estimators = 150, max_depth=5, random_state = 10)
X = train_data_numeric[decide_columns]
Y = train_data_numeric["Survived"]
dtree.fit(X, Y)

RandomForestClassifier(max_depth=5, n_estimators=150, random_state=10)

In [19]:
X_test = pd.get_dummies(test_data_numeric[decide_columns])
predictions = dtree.predict(X_test)
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('my_submission6.csv', index=False)